<a href="https://colab.research.google.com/github/annakim9237/Capstone_brainStation/blob/main/A1_Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Name**:

**Collaborators**:

# [CSC-496/696] Assignment 1, Problem 1: K-Means Clustering

### Total Points: 70 points for undergraduate students, 80 points for graduate students

The K-means algorithm clusters data by separating a dataset into $k$ groups. In other words, it divides a set of $N$ observations into $k$ disjoint clusters $C$. Each cluster of $C$ is described by mean $\mu_j$ of the observations that make up each cluster. These means $\mu_j$ are known as centroids.

To do this, the K-means clustering algorithm aims to estimate centroids that minimize the within-cluster variance.

To be clear, the inputs are:
* $N$ data points: these are observations in *multidimensional* space, and each data point is represented by a vector. Each data point is of dimension $d$.
* $k$ clusters: the number of clusters to form. $k$ is chosen by the researcher.
* K-means clustering is an *unsupervised* method, meaning we don't need ground truth labels to estimate the clusters

The algorithm proceeds as follows:
1. Initialization: we randomly select $k$ data points from the data set as initial centroids (mean vectors) of the cluster
2. Assignment Step: for each data point, we compute the Euclidean distance to each centroid. We then assign each data point to the nearest centroid. This forms $k$ clusters based on proximity to each centroid.
3. Update Step: we then calculate the new centroids by taking the mean of all data points assigned to each cluster. The centroid of each cluster is then updated to reflect the average position of the points within that cluster.
4. Iteration: we then repeat the assignment steps and update steps until we reach *convergence*. Convergence is when the centroids no longer change significantly between iterations. We can also set a max number of iterations.

In this problem, we'll be implementing K-means clustering.

## Problem 1.1: Reading in Data [10 points]

We'll be using data from the BuddyMove Data set. In this dataset, destination reviews were published by 249 reviewers on holidayiq.com until October 2014. Reviews that fell into 6 categories across destinations in South India were used and the count of reviews in each category for every reviewer was recorded. Because there are 6 categories, that means each datapoint is 6-dimensional.

Attribute 1 is the unique User ID. Attribute 2 is the number of reviews on stadiums and sport complexes. Attribute 3 is the number of reviews on religious institutions. Attribute 4 is the number of reviews on beaches, lakes, rivers, etc. Attribute 5 is the number of reviews on theaters, exhibitions, etc. Attribute 6 is the number of reviews on malls, shopping places, etc. And Attribute 7 is the number of reviews on parks, picnic spots, etc.

Like in lecture, we'll download it using the `gdown` function.

In [1]:
!gdown 1j3S-2iLjYU_A7z5k9KWOYU_6zvAoIxbu

Downloading...
From: https://drive.google.com/uc?id=1j3S-2iLjYU_A7z5k9KWOYU_6zvAoIxbu
To: /content/buddymove_holidayiq.csv
100% 7.57k/7.57k [00:00<00:00, 12.9MB/s]


In [2]:
import numpy as np
import pandas as pd

### Task 1.1.1: Question about Location of Dataset [3 points]

**Question**: Where is the file currently saved? What is the name of the file? In other words, type out what you would click on in Google Colab to see where the file currently is. Screenshots are optional, but welcomed.

**Answer**: The data file saved from google drive to directory of Google Colab "/content/" buddymove_holidayiq as csv file. The file name is buddymove_holidayiq.csv.

We can find the file via folder button, on left side in colab.

![Colab folder view](A1_problem1.png)



Now, let's read in the data so we can use it.

In [3]:
df = pd.read_csv('buddymove_holidayiq.csv')

### Task 1.1.2: Looking at the Data [3 points]

Use one line of code to look at the first 20 rows of the dataset.

In [5]:
# Implement your solution here
df.head(20)

,User Id,Sports,Religious,Nature,Theatre,Shopping,Picnic
0,User 1,2,77,79,69,68,95
1,User 2,2,62,76,76,69,68
2,User 3,2,50,97,87,50,75
3,User 4,2,68,77,95,76,61
4,User 5,2,98,54,59,95,86
5,User 6,3,52,109,93,52,76
6,User 7,3,64,85,82,73,69
7,User 8,3,54,107,92,54,76
8,User 9,3,64,108,64,54,93
9,User 10,3,86,76,74,74,103


### Task 1.1.3: Converting Pandas Dataframe to NumPy Array [4 points]

Using the `to_numpy()` method in the pandas module (described [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_numpy.html)), convert the data to an array but DO NOT include the first column, `User Id`. Call this dataset `reviews_data`.

HINT: You can select columns of a pandas dataframe using `dataset[['col1', 'col2', 'col3']]`, where the variable1 to variable3 are column names.

In [14]:
# Implement your solution here
df_num = df.loc[:, "Sports":"Picnic"].to_numpy()
df_num

array([[  2,  77,  79,  69,  68,  95],
       [  2,  62,  76,  76,  69,  68],
       [  2,  50,  97,  87,  50,  75],
       ...,
       [ 20, 124, 178, 104, 158, 174],
       [ 20, 133, 149, 139, 144, 213],
       [ 20, 143, 149, 139, 159, 143]])

## Problem 1.2: Implementing the Initialization [10 points]

We'll now implement choosing the initial centroids. `X` is our data and `k` is the number of clusters.

The first step of k-means clustering is the initialization step. Rather than just randomly choosing points, we'll just choose $k$ initial centroids from the dataset.

### Task 1.2.1: Implementing the Initialization [10 points]

Your task is to use `np.random.choice` to randomly choose $k$ rows as the centroids. Then, return those rows. Remember to sample without replacement.

To help you get started, `num_observations` is the number of observations in the dataset. It is the number of rows in data `X`.

In [16]:
def initialize_centroids(X, k):
    """Randomly initialize k centroids from the dataset X"""
    num_observations = X.shape[0]
    print(num_observations)
    np.random.choice(num_observations,k,replace=False)
    return

initialize_centroids(df_num,5)


249


### Task 1.2.2: Checking Your Implementation

After you have implemented the above, run the code below without modification. You should see an output of an array, with the first column reading `[14, 3, 6, 8, 14]`. The last column should read `[128, 69, 128, 118, 153]`.

There is nothing to do in this part but to check that your centroids are correct in this step.

In [ ]:
# Run code below without modification to check your implementation
np.random.seed(42)
trying_out_centroids = initialize_centroids(X=reviews_data, k=5)
print(trying_out_centroids)

## Problem 1.3: Implementing the Assignment Step [20 points]

The next step is to assign clusters to each datapoints. The cluster assigned to a datapoint is the centroid closest to the datapoint. To do this, we'll calculate the Euclidean distance of each datapoint to each centroid, and choose the centroid closest (lowest Euclidean distance).

The Euclidean distance is the distance between two points in Euclidean space, measured by the length of a line segment between them. It is calculated using the Pythagoras Theorem using the Cartesian coordinates of the points.

As an example, let $x$ and $y$ be two points in three-dimensional space. The Cartesian coodinates of $x$ is $(x_1, x_2, x_3)$ and the Cartesian coordinates of $y$ is $(y_1, y_2, y_3)$. Then, the Euclidean distance between them is:

$$d(x,y) = \sqrt{(x_1 - y_1)^2 + (x_2 - y_2)^2 + (x_3 - y_3)^2}$$

In general, for points given in Cartesian coordinates in $n$-dimensional Euclidean space, the distance is calculated as:

$$d(x,y) = \sqrt{(x_1 - y_1)^2 + (x_2 - y_2)^2 + (x_3 - y_3)^2 + ... + (x_{n-1} - y_{n-1})^2 + (x_n - y_n)^2}$$

### Task 1.3.1: Implementing Euclidean Distance [5 points]

Implement the Euclidean distance for any two vectors, $x$ and $y$ of any dimensions. Use only `np.sqrt()` and `np.sum()`. Do not use other functions.

In [ ]:
def euclidean_distance(x, y):
    """Calculate the Euclidean distance between two vectors, x and y"""

    # Delete pass and implement your solution here
    pass

You can check your implementation by running the code below. It should print out `True`.

In [ ]:
x = np.array([1,2,3])
y = np.array([4,5,6])
print(euclidean_distance(x,y)==5.196152422706632)

### Task 1.3.2: Calculating the Distance Matrix [5 points]

Next, we'll calculate the distance matrix. The distance matrix should be an array of size $(N, k)$ where $N$ is the total number of observations and $k$ is the number of centroids. Element $(i,j)$ will be the distance between observation $i$ and centroid $j$.

To implement this, you can use either loops with array indexing or you can use a non-loop version. The function should return the distance matrix. You don't have to use the `distance_matrix` array initialized for you in the function. In fact, with proper broadcasting, the non-loop version can be implemented in one line!

**HINT**: With the loop version, you might want to use the `euclidean_distance` function we just implemented. With the non-loop version, you'll have to use another approach to get the broadcasting just right.

In [ ]:
def calculate_distance_matrix(X, centroids):
    distance_matrix = np.zeros((X.shape[0], centroids.shape[0]))

    # Delete pass and implement your solution here
    pass

You can check your implementation by running the code below. It should print out 4 lines, each with "True"

In [ ]:
# Run code below without modification to check your implementation
trying_out_distance_matrix = calculate_distance_matrix(X=reviews_data, centroids=trying_out_centroids)
print(trying_out_distance_matrix.shape==(249,5))
print(trying_out_distance_matrix[0,0]==112.58330249197702)
print(trying_out_distance_matrix[1,4]==154.3243337908834)
print(trying_out_distance_matrix[4,4]==163.43500237097317)

### Task 1.3.3: Find the centroid closest to each data point [5 points]

Lastly, we'll choose the centroid that is closest to each data point. To do this, we'll implement `select_centroid`. Because we have the distance matrix now, we know which centroid the data point is closest to. We just need to find the index that has the lowest distance for each data point.

To do this, use `np.argmin`. [Here](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html) is the reference guide for this function. Don't forget to correctly choose which axis! **This should be a one-line solution.**

In [ ]:
def select_centroid(distance_matrix):
    """Select the centroid closest to each data point"""

    # Delete pass and implement your solution here
    pass

### Task 1.3.4: Putting it together [5 points]

In this last step, we'll put the above functions, `calculate_distance_matrix()` and `select_centroid()`, together in a single function. The function, `assignment_step`, should return the selected centroids for each observation. It should be an array of dimension $N$, where $N$ is the total number of observations.

In [ ]:
def assignment_step(X, centroids):
    """Perform the assignment step"""

    # Delete pass and implement your solution here
    pass

You can check your implementation by running the code below. It should print out 5 lines, each with "True"

In [ ]:
# Run code below without modification to check your implementation
trying_out_labels = assignment_step(reviews_data, trying_out_centroids)
print(sum(trying_out_labels==0)==77)
print(sum(trying_out_labels==1)==51)
print(sum(trying_out_labels==2)==40)
print(sum(trying_out_labels==3)==46)
print(sum(trying_out_labels==4)==35)

## Problem 1.4: Implementing the Update Step [10 points]

In this part, we'll implement the update step. After finding the closest centroid for each datapoint, we'll re-calculate each centroid by taking the mean (average) of all the datapoints assigned to a given centroid.

The intuition can be thought of as the following: imagine you're running an event with multiple ticketing booths, and these ticketing booths can be moved (the centroids). People (the data points) entering the event go to the nearest ticketing booth. But you notice that some people barely have to walk to get to a ticketing booth, while others have to walk quite a long distance to reach the others. So you move the ticketing booths so people have to walk less (update step). This is essentially what updating the centroids is---even though people entering the park still enter from the same places, you can move the ticketing booths so people, on average, are closer to the ticketing booths.

You don't have to use the `new_centroids` array, but it is there if you want to use it with a loop.

In [ ]:
def update_step(X, labels, k):
    """Perform the update step"""
    new_centroids = np.zeros((k, X.shape[1]))

    # Delete pass and implement your solution here
    pass

## Problem 1.5: Implementing K-Means [20 points]

We now have all the ingredients to put together k-means. As a reminder, this is how the algorithm works:

1. Initialization: we randomly select $k$ data points from the data set as initial centroids (mean vectors) of the cluster
2. Assignment Step: for each data point, we compute the Euclidean distance to each centroid. We then assign each data point to the nearest centroid. This forms $k$ clusters based on proximity to each centroid.
3. Update Step: we then calculate the new centroids by taking the mean of all data points assigned to each cluster. The centroid of each cluster is then updated to reflect the average position of the points within that cluster.
4. Iteration: we then repeat the assignment steps and update steps until we reach *convergence*. Convergence is when the centroids no longer change significantly between iterations. We can also set a max number of iterations.

### Task 1.5.1: Putting it all together [15 points]

Using the previous functions, implement k-means. We have a `max_iter` argument because we don't want it to run forever in case there is no convergence. `tol` sets how close we want the centroids to be before we say they're "close enough." Checking for convergence has already been implemented for you, assuming you correctly implemented the `euclidean_distance` function.

In [ ]:
def kmeans(X, k, max_iter=100, tol=1e-6):
    """Perform the k-means algorithm"""

    # Initialization of Centroids -- delete None and implement your solution here
    centroids = None

    # Iteration over the Assignment Step and Update Step
    for i in range(max_iter):
        # Assignment step -- delete None and implement your solution here
        labels = None

        # Update Step -- delete None and implement your solution here
        new_centroids = None

        # Check for Convergence
        if euclidean_distance(centroids, new_centroids) < tol:
            break

        # Update centroids
        centroids = new_centroids

    return centroids, labels

### Task 1.5.2: Question about `break` [5 points]

**Question**: What does the `break` do in the "# Check for Convergence" section of the code?

**Answer**:

### Task 1.5.3: Checking Your Implementation

Run the code snippet below, which uses the reviews data with 4 centroids. You should get the following 4 centroids:

* `[ 17.6744186   96.86046512 195.72093023 132.58139535  90.30232558
  147.44186047]`
* `[  5.82727273  87.44545455 101.85454545  97.68181818  85.85454545
   94.48181818]`
* `[ 15.59459459 163.32432432  82.94594595 107.89189189 183.05405405
  137.89189189]`
* `[ 17.06779661 127.25423729 140.94915254 144.74576271 134.69491525
  138.05084746]`

In [ ]:
# Run code below without modification to check your implementation
np.random.seed(20016)
centroids, labels = kmeans(X=reviews_data, k=4)
print('Centroids:\n')
print(centroids)
print('\nLabels:\n')
print(labels)

## Problem 1.6: Exploring K-Means [10 points]

**This problem only needs to be completed by graduate students, but undergraduate students are free to try it out to visualize what is happening with k-means.**

Run k-means with different values for $k$. Be sure to set the seed using `np.random.seed()` in order to make sure that your answers are reproducible.

There is a function below called `plot_kmeans_clusters`. This will use principal component analysis (PCA) to plot the reviews data and the centroids. Because our data is 6-dimensional, we need a way to reduce the dimensionality down to two dimensions (or three dimensions) to visualize it.

You don't need to understand what PCA is to visualize your data, but if you want to read more about it, read an introduction to PCA by IBM [here](https://www.ibm.com/topics/principal-component-analysis).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def plot_kmeans_clusters(X, final_centroids, final_labels):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    centroids_pca = pca.transform(final_centroids)

    plt.figure(figsize=(10, 7))

    unique_labels = set(final_labels)
    for label in unique_labels:
        plt.scatter(X_pca[final_labels == label, 0],
                    X_pca[final_labels == label, 1],
                    label=f'Cluster {label}')

    # Plot the centroids
    plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1],
                s=300, c='black', marker='X',
                label='Centroids')

    # Add title and labels
    plt.title('PCA Plot of K-means Clusters')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')

    # Add legend
    plt.legend()

    # Show the plot
    plt.show()

Here's an example on how to visualize the k-means results from the last problem, where we checked the implementation of k-means. Running this code, you should see a plot with colors corresponding to the 4 clusters, and X's marked for the centroid.

In [ ]:
np.random.seed(20016)
centroids, labels = kmeans(X=reviews_data, k=4)
plot_kmeans_clusters(X=reviews_data, final_centroids=centroids, final_labels=labels)

### Task 1.6.1: Choosing a value for $k$ [10 points]

**Question**: Try several values of $k$, and visualize the clustering using the code above. What value of $k$ seems optimal? Why? Note that there is no technical answer here. Do include plots and discuss why certain values of $k$ seem more optimal than others.

**Answer**: